# Homework 2
*Edited by Yibang (Christopher) Liu, 8/1*

## Reading
* Descriptor: https://realpython.com/python-descriptors/#how-attributes-are-accessed-with-the-lookup-chain
* Data model: https://docs.python.org/3/reference/datamodel.html (contents related to what we taught)
* Python MRO: https://www.python.org/download/releases/2.3/mro/

## @vectorize decorator

Implement a vector that can "vectorize" a callable. For example, given a function that calculates the log of the input floating number, the vectorize decorator will allow the decorated function to operate on an iterable of floating numbers, and returns a list of numbers.

* Do not worry about the performance. 
* You should allow your decorator to work with callables with multiple arguments. e.g. decorated version of `f(x: int, y: int)` should be able to take two sequences with same size.

Demonstrate what your decorator can do with some examples.

```python
import math


@vectorize
def log2(num):
    return math.log(num, 2)

>>> log2((1, 2, 3)) 
[0.0, 1.0, 1.5849625007211563]


@vectorize
def add(x, y):
    return x + y


>>> add([1, 2, 3], [4, 5, 6])
[5, 7, 9]
```

In [1]:
def vectorize(func):
    
    def inner(*nums):
        length = len(nums[0])
        # Can only work on iterables of the same length
        if all([len(num) == length for num in nums]):
            return [func(*(num[i] for num in nums)) for i in range(length)]
        raise ValueError("Different lengths.")
        
    return inner

In [2]:
import math

@vectorize
def log2(num):
    return math.log(num, 2)

log2((1, 2, 3))

[0.0, 1.0, 1.5849625007211563]

In [3]:
@vectorize
def add(x, y):
    return x + y

add([1, 2, 3], [4, 5, 6])

[5, 7, 9]

In [4]:
@vectorize
def multiply(x, y, z):
    return x * y * z

multiply([1, 2, 3], [4, 5, 6], [0, 2, 4])

[0, 20, 72]

In [5]:
multiply([1, 2], [4, 5, 6], [0, 2, 4])

ValueError: Different lengths.

## Merge sorted stream
Write a generator function that takes two sorted streams (generators), and return a generator that can produce a merged stream in sorted order.

Bonus point: can you make it generic such that it can merge any number of streams?


```python
def merge_sorted_stream(stream1, stream2):
    yield ...

stream1 = range(0, 10, 2)
stream2 = range(1, 10, 2)

for x in merge_sorted_stream(stream1, stream2):
    print(x)

0 
1
2
3
4
5
6
7
8
9
```

In [6]:
def merge_sorted_stream(*streams):
    lists = [list(stream) for stream in streams]
    sorted_list = sorted([l[i] for l in lists for i in range(len(l))])
    yield from sorted_list

In [7]:
stream1 = range(0, 10, 2)
stream2 = range(9, 0, -2)
stream3 = reversed([0, -30, 68])

for x in merge_sorted_stream(stream1, stream2):
    print(x)

0
1
2
3
4
5
6
7
8
9


In [8]:
for x in merge_sorted_stream(stream1, stream2, stream3):
    print(x)

-30
0
0
1
2
3
4
5
6
7
8
9
68


## Tree traversal

Unlike linear data structures (Array, Linked List, Queues, Stacks, etc) which have only one logical way to traverse them, trees can be traversed in different ways. Following are the generally used ways for traversing trees.

```
      1
    /  \ 
   2    3
  / \
 4   5
```

Depth First Traversals: 
  * (a) Inorder (Left, Root, Right) : 4 2 5 1 3
  * (b) Preorder (Root, Left, Right) : 1 2 4 5 3
  * (c) Postorder (Left, Right, Root) : 4 5 2 3 1

Define a Tree class with a method that can walk through the tree in different orders. Hint: use generator will make your life a lot easier.

```python
class TreeNode:
    
    ...

    def in_order(self):
        pass
        
    def pre_order(self):
        pass
        
    def post_order(self):
        pass

    
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

>>> print(' -> '.join(item for item in root.in_order()))
4 -> 2 -> 5 -> 1 -> 3    


```
   

In [9]:
class TreeNode:
    def __init__(self, val = 0, left = None, right = None):
        self.val = val
        self.left = left
        self.right = right

    def in_order(self):
        if self.left:
            yield from self.left.in_order()
        yield str(self.val)
        if self.right:
            yield from self.right.in_order()

    def pre_order(self):
        yield str(self.val)
        if self.left:
            yield from self.left.pre_order()
        if self.right:
            yield from self.right.pre_order()

    def post_order(self):
        if self.left:
            yield from self.left.post_order()
        if self.right:
            yield from self.right.post_order()
        yield str(self.val)

In [10]:
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

In [11]:
print(' -> '.join(item for item in root.in_order()))
print(' -> '.join(item for item in root.pre_order()))
print(' -> '.join(item for item in root.post_order()))

4 -> 2 -> 5 -> 1 -> 3
1 -> 2 -> 4 -> 5 -> 3
4 -> 5 -> 2 -> 3 -> 1


## Implement a timer
Implement a timer that can print the execution time of your code. Try to implement it both as a decorator and as a context manager to compare the implementations. Can you implement it using one single class? 

Example:
```python
import time


with timer() as timer:
    time.sleep(3)

# Total execution time: 3.000123 seconds (a made up number)


@timer
def sleep(secs):
    time.sleep(secs)
    
sleep(3)
# Total execution time: 3.000123 seconds (a made up number)
```

Below is the code snippet to measure the time:
```python    
import time
start_time = time.time()
main()
print(f"--- {time.time() - start_time} seconds ---")
```

In [12]:
import time

class timer:    
    def __call__(self, func):
        def inner(*args, **kwags):
            start_time = time.time()
            func(*args, **kwags)
            end_time = time.time()
            print(f"Total execution time: {end_time - start_time} seconds")
        return inner
    
    def __enter__(self):
        self.start = time.time()
        
    def __exit__(self, exc_type, exc_value, traceback):
        self.end = time.time()
        print(f"Total execution time: {self.end - self.start} seconds")
        return True

In [13]:
with timer() as t:
    time.sleep(3)

Total execution time: 3.012789249420166 seconds


In [14]:
@timer()
def sleep(secs):
    time.sleep(secs)
    
sleep(3)

Total execution time: 3.0043656826019287 seconds
